# Analysis of one layer of 3DXRD data
This notebook illustrates how to run 3DXRD analysis on one slice with the help of DDDXRD and Fable.

In [1]:
import dddxrd.peaksearch.median_image as mi
import dddxrd.peaksearch.peaksearching as ps
import dddxrd.indexing.indexing as idx
import dddxrd.visualization.Grainmap as visu
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np

## Peaksearching
The first step in the analysis is to search for diffraction peaks in all images. The peaks are identified based on the intensity values in the images by thresholding using one or more threshold values. The peaksearching is done using the peaksearch.py script from ImageD11. We use a .yaml file to pass parameters to the script. The result will be a bunch of *.flt and *.spt files (one for each threshold level) containing information about all identified peaks.

### Computing a background image
Before we can start looking for peaks we need to subtract the background. Typically there are a bunch of dark images in the dataset, so lets take the median of those and use as our background. The parameters for this are also set in a .yaml file.

In [2]:
# path to .yaml file containing parameter for median image
yaml_path = 'dddxrd/tests/median_image.yaml'
mimg = mi.median_image(yaml_path)
plt.figure()
plt.imshow(mimg,cmap=plt.cm.gray,vmin=np.mean(mimg)-2*np.std(mimg),vmax=np.mean(mimg)+2*np.std(mimg))

Looking for images in: dddxrd/tests
Found 2 images
Saved median image in /Users/al8720/code/DDDXRD/dddxrd/tests/median_dark.tif


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Finding diffraction spots
We are now ready to look for diffraction spots. The peaksearch will be distributed on all available cpus, so it shouldn't take too long. The will be a lot of printout and it will probably raise an exception but it seems to work nonetheless.

In [4]:
# path to .yaml file containing parameters for peaksearching
yaml_path = 'dddxrd/tests/peaksearch.yaml'
# run the peaksearch
ps.run_peaksearch(yaml_path)

Running peaksearch with the following command:
peaksearch.py -n dddxrd/tests/load_0N_ -F .tif -f 1 -l 2 -o dddxrd/tests/peaks/peaks -d dddxrd/tests/median_dark.tif -p Y --ndigits 5 -S -0.500 -T 358.560 -t 20000 -t 10000 -t 5000 --OmegaOverride 


### Merging the different thresholds
We can now merge the spots from the different threshold levels. This works by selecting all peaks from the highest threshold and adding peaks from the lower thresholds if they are not already present. For the peaks found in both levels, the lower threshold version is discared if the centre of mass position is not within pixel_tol pixels from the high threshold peak. We want to do this step to separate peaks that are close to eachother on the detector. This step will create a new .flt file containing the merged peaks.

In [5]:
yaml_path = 'dddxrd/tests/peaksearch.yaml'
ps.merge_peaks(yaml_path)

Merging flt files matching dddxrd/tests/peaks/peaks
merge_flt.py dddxrd/tests/junk dddxrd/tests/peaks/peaks dddxrd/tests/peaks/merged_peaks.flt 10 20000 10000 5000 


## Fitting the geometry and indexing the data
We are now ready to fit the geometrical parameters of the measurement, i.e., the distance to the detector, the beam center position, and the detector tilts. To do this we fire up the *ImageD11_gui* and load the merged .flt file. The process for fitting the geometry and doing a "manual" indexation of the data is described here: https://imaged11.readthedocs.io/en/latest/_downloads/dd7bdff1e8c819ee92e40c22efa75f21/com_guide.pdf
The most tricky thing in this step is to figure out the image flips (o11,o12,o21,o22), each of the paramter can take the values -1,0,1. These parameters descibes the mapping from the real world coordinates to the coordinates used by the detector when saving the data. There are 8 possible orientations. With a bit of luck, 4 of them can be eliminated by looking at the images in e.g. Fiji. What you are looking for is diffraction spots that are very long, i.e. does not dissapear when stepping through the images, in the rotation direction, they are most likely placed along the horizontal or the vertical axis of the images. The spots are long because the normal to that particular crystal plane is parallel to the rotation axis. Assuming that the rotation axis was vertical in the experiment; if the long spots are along the vertical direction you can deduce that o12=o21=0, i.e. the image is not rotated when saved. Similarly if the spots are on the horizontal axis o11=o22=0, i.e., the image is rotated 90 degrees in one direction when saved. There are now 4 possible combinations left. The best way I've found to find the correct one is to test them all and pick the one which gives the most convincing indexation. One useful thing to do is to 'write out indexed peaks' from the gui. This will create a text file containing the observed and computed angles for all indexed peaks.  

There is also some useful info at the TIMEleSS wiki: http://multigrain.texture.rocks/doku.php?id=dac_experiments:geometry

## Script based indexation
When you have the geometry and the indexation paramters figured out you can process the data faster by indexing and mapping the grains using a script. The script is indexing on a grid to also try to catch grains far from the center of rotation (the same as setting t_x,t_y,t_z parameter in the gui). The parameters are again set in a .yaml file. The output will be a .map file containing the orientation and position of each grain.

In [8]:
yaml_path = 'dddxrd/tests/indexing.yaml'
idx.index_and_map(yaml_path)


ed 24 5.5053506768438746e-05 1.4370438052035812
           matched 25 4.688764430130701e-05 1.314070654289557
           matched 26 2.8978038473417905e-05 1.3802225372663763
           matched 27 4.353408482428245e-05 0.824192597702003
           matched 28 3.810547785496551e-05 1.3437372021192096
           matched 29 1.8430439999894915e-05 2.727268243507939
           matched 30 6.71176871862331e-05 1.2422825134840338
           matched 31 1.698988104306101e-05 0.8742980576555913
           matched 32 7.907437466102674e-05 1.4755544850781066
           matched 33 4.886688901776624e-05 0.8460665979623574
           matched 34 1.8548711416805456e-05 1.1170300281798171
           matched 35 3.40226387745809e-05 1.01258887517043
           matched 36 0.0 0.768269130567162
           matched 37 0.0 0.8956644287914011
           matched 38 4.122940033309676e-05 0.998392063676677
Got    39 new 0 from 39
           matched 0 0.00010649264863120937 3.08899888426813
           matched 1 0.0001

## Visualization of the result

In [11]:
mapfile = 'alltest.map'
gm=visu.Grainmap(mapfile)
fig,ax = gm.plot_3d_map()

view = ('top','side','front')
cind = ([0,1],[0,2],[1,2])
lab = (['x','y'],['x','z'],['y','z'])

for v,c,l in zip(view,cind,lab):
    fig,ax = gm.scatterplot(np.array([gm.com[:,c[0]],gm.com[:,c[1]]]).T,c=gm.rgb,s=gm.size/2,alpha=0.6,edgecolor='k',marker='o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(39, 3, 2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …